In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


XOR 연산은 입력 데이터를 X 라고 할경우 [[0,0],[0,1],[1,0],[1,1]] 과 같은 경우의 수가 있다 .  
총 4개의 데이터가 각각 2개의 특성을 가지고 있으므로 X는 [4, 2]의 형태로 정의 할 수 있다.  
입력값에 따른 출력값 Y는 [[0], [1], [1], [0]]이므로 Y는 [4, 1]의 형태로 정의할 수 있다.

In [2]:
X = tf.placeholder(dtype=tf.float32, shape=[4,2]) # 입력값
Y = tf.placeholder(dtype=tf.float32, shape=[4,1]) # 출력값

1번째 히든 레이어에서 2개의 입력(X)과 1개의 편향값(B1)을 받아서 2개의 시그모이드 출력(Z)을 다음 레이어로 전달한다.

In [3]:
# 2개의 입력을 받는 2개의 뉴런을 만든다.  
# random_normal()
W1 = tf.Variable(tf.random_normal([2, 2]))
# 각 뉴런은 1개의 편향값을 가진다.
# zeros() 메소드는 모든 원소의 값이 0인 텐서(2차원 이상의 행렬)를 생성한다.
B1 = tf.Variable(tf.zeros([2]))
# 시그 모이드 함수를 거쳐 출력값으로 Z를 리턴한다 => tf.sigmoid()
Z = tf.sigmoid(tf.matmul(X, W1) + B1)

In [4]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
print(sess.run(W1))
print(sess.run(B1))
train_X = [[0, 0], [0, 1], [1, 0], [1, 1]]
print(sess.run(Z, feed_dict={X: train_X}))
sess.close()

[[ 0.14013813 -0.4958821 ]
 [ 0.56394535  0.41079688]]
[0. 0.]
[[0.5        0.5       ]
 [0.6373649  0.60127896]
 [0.5349773  0.3785089 ]
 [0.66909254 0.47874153]]


2번쨰 히든 레이어에서 1번째 히든 레이어의 출력 값인 Z 와 1개의 편향값을 받아서 1개의 시그모이드를 출력(y_hat)한다.

In [5]:
# Z를 입력값으로 받는 1개의 뉴런을 만든다. 
W2 = tf.Variable(tf.random_normal([2,1]))
B2 = tf.Variable(tf.zeros([1]))
# 시그 모이드 함수를 거쳐 출력값으로 Z를 리턴한다. => tf.sigmoid(W2 * Z + B2)
Y_hat = tf.sigmoid(tf.matmul(Z, W2) + B2)

In [6]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
print(sess.run(W2))
print(sess.run(B2))
print(sess.run(Y_hat, feed_dict={X: train_X}))
sess.close()

[[ 0.70742357]
 [-0.6034921 ]]
[0.]
[[0.5129885]
 [0.5395108]
 [0.5286413]
 [0.5477651]]


손실 함수로크로스 엔트로피를 사용하고 경사 하강법으로 모델의 매개 변수(가중치, 편향)을 최적화한다.

In [10]:
# 크로스 엔트로피
loss = tf.reduce_mean(
    -1 * (Y * tf.log(Y_hat) + (1.0 - Y) * tf.log(1.0 - Y_hat))
)
# 경사 하강법
train_step = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
# 모델을 학습하기 위한 데이터를 만든다.
train_X = [[0,0],[0,1],[1,0],[1,1]]
train_Y = [[0],[1],[1],[0]]

학습을 진행한다.

In [16]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print('학습 데이터: {}'.format(train_X))
    print('='*80)
    
    for epoch in range(50001):
        sess.run(train_step, feed_dict={X: train_X,Y:train_Y})
        if epoch % 5000 == 0:
            print('epoch: {}'.format(epoch))
            print('output\n',sess.run(Y_hat,feed_dict={X:train_X, Y:train_Y}), sep='')
            print('='*80)

학습 데이터: [[0, 0], [0, 1], [1, 0], [1, 1]]
epoch: 0
output
[[0.53617847]
 [0.63918656]
 [0.44028157]
 [0.5471509 ]]
epoch: 5000
output
[[0.36597103]
 [0.8508332 ]
 [0.37187284]
 [0.35126048]]
epoch: 10000
output
[[0.08561283]
 [0.9020924 ]
 [0.88641024]
 [0.07226974]]
epoch: 15000
output
[[0.03816473]
 [0.95250845]
 [0.9498846 ]
 [0.03318483]]
epoch: 20000
output
[[0.02407479]
 [0.9693779 ]
 [0.96833074]
 [0.02111778]]
epoch: 25000
output
[[0.01747382]
 [0.97753745]
 [0.9769659 ]
 [0.01539919]]
epoch: 30000
output
[[0.01367426]
 [0.98230565]
 [0.98194075]
 [0.01208818]]
epoch: 35000
output
[[0.01121488]
 [0.9854219 ]
 [0.98516613]
 [0.00993684]]
epoch: 40000
output
[[0.00949675]
 [0.9876139 ]
 [0.9874228 ]
 [0.00842974]]
epoch: 45000
output
[[0.00822982]
 [0.98923784]
 [0.9890888 ]
 [0.00731629]]
epoch: 50000
output
[[0.00725734]
 [0.99048805]
 [0.9903672 ]
 [0.00646007]]


학습 결과를 통해 [0, 0], [1, 1]은 상당히 가까운 값을 출력하고 [0, 1], [1, 0]은 1에 상당히 가까운 값을 출력하는것을 확인할 수 있다.

<img src="./images/differ1.png" align="left">

In [17]:
def diff1(f, x, h=1e-6):
    pass

미분(Differential) 함수 정의: 대칭 파생(Symmetric derivative)  
diff2(미분할 함수, 미분할값Δx)

<img src="./images/differ2.png" align="left">